In [29]:
from glob import glob
from pathlib import Path
import ROOT
import pandas as pd
from array import array
import csv
# import matplotlib.pyplot as plt

In [44]:
absolute_path_only = '/media/droc/Seagate/PHYS-461/6-compton/data/' #This will need to change depending on the path to data
input_files = glob(absolute_path_only + '/*.csv')

angle = []
base_file = []
for file in input_files:
    base = Path(file).stem
    base_file.append(base)
    i = base.find('-')
    angle.append(base[:i])

base_file.sort()
angle.sort()
# print(angle)
# print(len(angle))
# print(base_file)

In [31]:
input_file = glob(absolute_path_only + '/C*.csv')
print(input_file)

['/media/droc/Seagate/PHYS-461/6-compton/data/COMPTON-DATA.csv']


In [ ]:
# .csv -> .root

def clean_data(df):
        for col in df.columns:
            # Convert all values in the column to float, set non-convertible values to NaN
            df[col] = pd.to_numeric(df[col], errors='coerce')
        return df



df = pd.read_csv(input_file[0])

file_out_root = input_file[0].replace('.csv','.root')

root_file = ROOT.TFile(file_out_root, "RECREATE")
tree = ROOT.TTree("data", "Data from CSV")

branches = {}

# df = clean_data(df)

for column in df.columns:
    branches[column] = array('f', [0])  # Create float arrays for each column
    tree.Branch(column, branches[column], f'{column}/F')

for index, row in df.iterrows():
    for column in df.columns:
        branches[column][0] = row[column]
    tree.Fill()

tree.Write()
root_file.Close()


In [62]:
line = ROOT.TF1('line', '[0]*x+[1]', 0, 11)

c0 = ROOT.TCanvas("c0","", 800,800,1000,550)

file = ROOT.TFile(file_out_root) 
tree = file.Get("data")
rdf = ROOT.RDataFrame(tree)

cos_1 = rdf.AsNumpy(columns=["1-cos(theta)"])["1-cos(theta)"]
E_1 = rdf.AsNumpy(columns=["1-E'"])["1-E'"]
stddev_1_E = rdf.AsNumpy(columns=["sigma_1-E'"])["sigma_1-E'"]
ang_rad = rdf.AsNumpy(columns=["rangle(radians)"])["rangle(radians)"]


Error in <TUnixSystem::WorkingDirectory>: getcwd() failed
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c0


In [63]:
c0.Update()

gr = ROOT.TGraphErrors(len(cos_1),cos_1,E_1,0,stddev_1_E)
gr.SetMarkerStyle(ROOT.kFullCircle)
gr.SetTitle("1/E' vs [1-cos(#theta)]")
gr.GetXaxis().SetTitle("[1-cos(#theta)]")
gr.GetYaxis().SetTitle("1/E'")
gr.Fit(line, "Q")

p0 = line.GetParameter(0)
p0_error = line.GetParError(0)
p1 = line.GetParameter(1)
p1_error = line.GetParError(1)

gr.Draw()

latex = ROOT.TLatex()
latex.SetNDC()
latex.SetTextSize(0.06)
latex.DrawLatex(0.4,0.8,f'#scale[0.6]{{#bf{{slope: {p0:.2e} #sigma = #pm {p0_error:.2e} }}}}') 
latex.DrawLatex(0.4,0.76,f'#scale[0.6]{{#bf{{y-intercept: {p1:.2e} #sigma = #pm {p1_error:.2e}}}}}') 



png_file = file_out_root.replace(".root",".pdf")
c0.SaveAs(png_file)
c0.Clear()


Info in <TCanvas::Print>: pdf file /media/droc/Seagate/PHYS-461/6-compton/data/COMPTON-DATA.pdf has been created
